In [ ]:
import argparse
from typing import Union, List

import cv2
import numpy
import scipy
from matplotlib import pyplot as plt
from matplotlib.patches import Circle
from scipy import ndimage

from demo import Demonstration
from ply import write_xyz_rgb_as_ply, Ply
from trainer import Trainer

%matplotlib inline

In [ ]:
def read_image(path: Union[str, List[str]], img_type: str, history=False):
    """
    Reads image into numpy array
    @param path: Path to image
    @param img_type: One of 'color', 'depth'
    @param history: Whether or not to read history for depth images
    @return: Array containing image contents
    """
    # This is repeated several times in the code and should ideally be refactored into a function

    if img_type == "color":
        return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
    elif img_type == "depth":
        if history:
            return numpy.stack([cv2.imread(file, -1).astype(numpy.float32)/100000 for file in path], axis=-1)
        else:
            return numpy.stack([cv2.imread(path, -1)]*3, axis=-1).astype(numpy.float32)/100000
    return None

In [ ]:
# Workspace limits
workspace_limits = numpy.asarray([[-0.724, -0.276], [-0.224, 0.224], [-0.0001, 0.5]])

# Test images
test_color_heightmap = read_image("logs/2020-06-07-16-31-33_Real-Stack-Two-Step-Reward-Masked-Testing/data/color-heightmaps/000004.0.color.png", 'color')
test_depth_heightmap = read_image(["logs/2020-06-07-16-31-33_Real-Stack-Two-Step-Reward-Masked-Testing/data/depth-heightmaps/000004.0.depth.png",
                                   "logs/2020-06-07-16-31-33_Real-Stack-Two-Step-Reward-Masked-Testing/data/depth-heightmaps/000003.0.depth.png",
                                   "logs/2020-06-07-16-31-33_Real-Stack-Two-Step-Reward-Masked-Testing/data/depth-heightmaps/000002.0.depth.png"], 'depth', True)

print(test_depth_heightmap.shape)

In [ ]:

stack_snapshot_file = 'logs/base_models/rows_hist_densenet/snapshot.reinforcement_action_efficiency_best_value.pth'
# stack_snapshot_file = 'logs/base_models/stacking_hist_densenet/snapshot.reinforcement_action_efficiency_best_value.pth'
# stack_snapshot_file = 'logs/base_models/unstacking_hist_densenet/snapshot.reinforcement_action_efficiency_best_value.pth'
# stack_snapshot_file = 'logs/base_models/vertical_square_hist_densenet/snapshot.reinforcement_trial_success_rate_best_value.pth'
policy_name = 'row'
# policy_name = 'stack'
# policy_name = 'unstack'
# policy_name = 'square'

In [ ]:
# Demo files
demo = Demonstration("logs/demos/vertical_square_demos", 1, None)
demo_color_heightmap, demo_depth_heightmap = demo.get_heightmaps("grasp", 12, use_hist=True)



stack_trainer = Trainer(method='reinforcement', push_rewards=True, future_reward_discount=0.5,
                        is_testing=True, snapshot_file=stack_snapshot_file,
                        force_cpu=False, goal_condition_len=0, place=True,
                        pretrained=True, flops=False, network='densenet',
                        common_sense=True, place_common_sense=True,
                        show_heightmap=False, place_dilation=0.01,
                        common_sense_backprop=True, trial_reward='spot',
                        num_dilation=0)

In [ ]:
# get demo action index vector
action_vector = demo.action_dict[3][1]

# convert rotation angle to index
best_rot_ind = numpy.around((numpy.rad2deg(action_vector[-2]) % 360) * 16 / 360).astype(int)
# test_rot_ind = 15

# convert robot coordinates to pixel
workspace_pixel_offset = workspace_limits[:2, 0] * -1 * 1000
best_action_xy = ((workspace_pixel_offset + 1000 * action_vector[:2]) / 2).astype(int)

print(best_action_xy)
print(best_rot_ind)

In [ ]:
_, demo_features, _ = stack_trainer.forward(demo_color_heightmap, demo_depth_heightmap,
                                            is_volatile=True, keep_action_feat=True, demo_mask=True)[:3]
demo_features = demo_features.filled(0.0)

In [ ]:
_, test_features , _ = stack_trainer.forward(test_color_heightmap, test_depth_heightmap,
                                            is_volatile=True, keep_action_feat=True, demo_mask=True)[:3]
test_features = test_features.filled(0.0)


In [ ]:
demo_features_rot = demo_features[best_rot_ind,:,:,:]
# test_features_rot = test_features[test_rot_ind,:,:,:]

# Compute mask
demo_mask = (demo_features_rot == 0).all(axis=0)
# test_mask = (test_features_rot == 0).all(axis=0)
test_mask = (test_features==0).all(axis=1)

In [ ]:
# match_dist = numpy.zeros([16, 224, 224])
match_dist = numpy.sum(numpy.square(numpy.expand_dims(demo_features_rot[:, best_action_xy[1], best_action_xy[0]],(0,2,3)) - test_features), axis=1)
match_dist[test_mask] = numpy.max(match_dist)
matched_action = numpy.unravel_index(numpy.argmin(match_dist), (16, 224, 224))

test_rot_ind = matched_action[0]
test_features_rot = test_features[test_rot_ind,:,:,:]
test_mask_rot = test_mask[test_rot_ind,:,:]

In [ ]:
fig, ax = plt.subplots(1)
ax.imshow(match_dist[test_rot_ind,:,:], cmap="gray")

circle = Circle((matched_action[2], matched_action[1]))
ax.add_patch(circle)
print(matched_action)

In [ ]:
rematch_squared_distance = numpy.zeros([224, 224])

for i in range(0, 224):
    for j in range(0, 224):
        x = numpy.expand_dims(test_features_rot[:,i,j], (1,2))
        displacement = demo_features_rot - x
        distances = numpy.sum(numpy.square(displacement), axis=0)
        distances = ndimage.gaussian_filter(distances, sigma=(3,3))
        distances[demo_mask] = numpy.max(distances) * 1.1
        match_index = numpy.unravel_index(numpy.argmin(distances), (224, 224))
        rematch_squared_distance[i,j] = numpy.sum(numpy.square(match_index - best_action_xy[[1, 0]]))

rematch_distance = numpy.sqrt(rematch_squared_distance)

In [ ]:
fig, ax = plt.subplots(1)

ax.imshow(demo_color_heightmap)
circle = Circle(best_action_xy)
ax.add_patch(circle)
# fig, ax = plt.imshow(demo_color_heightmap)


In [ ]:
plt.imshow(test_color_heightmap)

In [ ]:
plt.imshow(demo_mask, cmap='gray')

In [ ]:
plt.imshow(test_mask_rot, cmap='gray')

In [ ]:
rematch_distance[test_mask_rot] = numpy.max(rematch_distance)
plt.imshow(rematch_distance, cmap='gray')

In [ ]:
inverse_heights = numpy.max(rematch_distance) - rematch_distance
inverse_heights[test_mask_rot] = 0
# inverse_heights[~test_mask] = 200



In [ ]:
inverse_heights_scaled = (inverse_heights - numpy.mean(inverse_heights))/numpy.std(inverse_heights)


In [ ]:

inverse_heights_image = cv2.applyColorMap(((1-inverse_heights/numpy.max(inverse_heights))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
plt.imshow(inverse_heights_image)

inverse_heights_image_reverse = cv2.applyColorMap(((inverse_heights/numpy.max(inverse_heights))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
cv2.imwrite("figures/fig7/inverse_heights_" + policy_name + ".png", inverse_heights_image_reverse)
# cv2.imwrite("figures/fig7/inverse_heights_stack.png", inverse_heights_image_reverse)
# cv2.imwrite("figures/fig7/inverse_heights_unstack.png", inverse_heights_image_reverse)
# cv2.imwrite("figures/fig7/inverse_heights_square.png", inverse_heights_image_reverse)
blended = cv2.addWeighted(inverse_heights_image_reverse, 0.5, test_color_heightmap, 0.5, 0)
cv2.imwrite("figures/fig7/inverse_heights_blended_" + policy_name + ".png", blended)

In [ ]:

# inverse_heights_softmax = numpy.log(numpy.exp(inverse_heights_scaled)/numpy.sum(numpy.exp(inverse_heights_scaled)))
inverse_heights_softmax = scipy.special.softmax(inverse_heights_scaled)
inverse_heights_softmax = (inverse_heights_softmax - numpy.min(inverse_heights_softmax))/(numpy.max(inverse_heights_softmax-numpy.min(inverse_heights_softmax)))
inverse_heights_image_softmax = cv2.applyColorMap(((inverse_heights_softmax/numpy.max(inverse_heights_softmax))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
cv2.imwrite("figures/fig7/inverse_heights_softmax_" + policy_name + ".png", inverse_heights_image_softmax)
blended = cv2.addWeighted(inverse_heights_image_softmax, 0.5, test_color_heightmap, 0.5, 0)
cv2.imwrite("figures/fig7/inverse_heights_softmax_blended_" + policy_name + ".png", blended)
inverse_heights_image_softmax = cv2.applyColorMap(((1-inverse_heights_softmax/numpy.max(inverse_heights_softmax))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
plt.imshow(inverse_heights_image_softmax)

In [ ]:
print(numpy.max(inverse_heights_softmax))
print(numpy.min(inverse_heights_softmax))


In [ ]:
# inverse_heights_softmax = numpy.log(numpy.exp(inverse_heights_scaled)/numpy.sum(numpy.exp(inverse_heights_scaled)))
inverse_heights_log_softmax = scipy.special.log_softmax(inverse_heights_scaled)

inverse_heights_log_softmax = (inverse_heights_log_softmax - numpy.min(inverse_heights_log_softmax))/(numpy.max(inverse_heights_log_softmax-numpy.min(inverse_heights_log_softmax)))
# inverse_heights_avg_softmax_log_softmax = inverse_heights_softmax + inverse_heights_log_softmax * (inverse_heights_log_softmax < 0.5)
inverse_heights_avg_softmax_log_softmax = inverse_heights_softmax + inverse_heights_log_softmax
inverse_heights_avg_softmax_log_softmax_image = cv2.applyColorMap(((inverse_heights_avg_softmax_log_softmax/numpy.max(inverse_heights_avg_softmax_log_softmax))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
cv2.imwrite("figures/fig7/inverse_heights_avg_softmax_log_softmax_" + policy_name + ".png", inverse_heights_avg_softmax_log_softmax_image)
blended = cv2.addWeighted(inverse_heights_avg_softmax_log_softmax_image, 0.5, test_color_heightmap, 0.5, 0)
cv2.imwrite("figures/fig7/inverse_heights_avg_softmax_log_softmax_blended_" + policy_name + ".png", blended)
inverse_heights_avg_softmax_log_softmax_image = cv2.applyColorMap(((1-inverse_heights_avg_softmax_log_softmax/numpy.max(inverse_heights_avg_softmax_log_softmax))*255).astype(numpy.uint8), cv2.COLORMAP_JET)
plt.imshow(inverse_heights_avg_softmax_log_softmax_image)

In [ ]:
print(numpy.shape(inverse_heights_avg_softmax_log_softmax))

In [ ]:
plt.imshow(test_color_heightmap)

In [ ]:
# Compute colors
# colors = [test_color_heightmap[i,j,:]/255 for j in range(0, 224) for i in range(0, 224)]
x = [i for j in range(0, 224) for i in range(0, 224)]
y = [j for j in range(0, 224) for i in range(0, 224)]
# top = [inverse_heights[i,j] for j in range(0, 224) for i in range(0, 224)]
# top = [inverse_heights_softmax[i,j] for j in range(0, 224) for i in range(0, 224)]
top = [inverse_heights_avg_softmax_log_softmax[i,j] for j in range(0, 224) for i in range(0, 224)]

best_test_action_index = numpy.argmax(top)
best_test_action = (y[best_test_action_index], x[best_test_action_index])
# inverse_heights_image = cv2.circle(inverse_heights_image, best_test_action, 2, (255, 0, 0), 2)
# blended = cv2.addWeighted(inverse_heights_image, 0.25, test_color_heightmap, 0.75, 0)
# blended = cv2.addWeighted(inverse_heights_image_softmax, 0.5, test_color_heightmap, 0.5, 0)
blended = cv2.addWeighted(inverse_heights_avg_softmax_log_softmax_image, 0.5, test_color_heightmap, 0.5, 0)


blended = cv2.circle(blended, best_test_action, 2, (255, 0, 0), 2)
plt.imshow(blended)

colors = [blended[i,j,:]/255 for j in range(0, 224) for i in range(0, 224)]
# _x = numpy.arange(224)
# _y = numpy.arange(224)
# _xx, _yy = numpy.meshgrid(_x, _y)
# x, y = _xx.ravel(), _yy.ravel()

# top = inverse_heights.flatten('C')
bottom = numpy.zeros(len(top))

In [ ]:
figure = plt.figure(figsize=(6, 4), dpi=600)
ax = figure.add_subplot(111, projection='3d')
ax.view_init(elev=15., azim=90)
# ax.view_init(elev=30., azim=0)

# ax.plot_surface(numpy.array(x), numpy.array(y), numpy.array(top), color=colors)
ax.bar3d(x, y, bottom, 1, 1, top, shade=False, color=colors)

# ax.set_zlim(0, 500)
ax.set_zlim(0, numpy.max(top)*5)

plt.axis('off')

plt.show()

In [ ]:
figure.savefig("figures/fig7/test_cc_figure_7_" + policy_name + ".png", transparent=True)
# figure.savefig("figures/fig7/test_cc_figure_7_stack.png", transparent=True)
# figure.savefig("figures/fig7/test_cc_figure_7_unstack.png", transparent=True)
# figure.savefig("figures/fig7/test_cc_figure_7_square.png", transparent=True)

In [ ]:
masked_test_color_heightmap = test_color_heightmap.copy()
masked_test_color_heightmap[test_mask_rot,:] = 0
# best_test_action = numpy.unravel_index(numpy.argmin(rematch_distance), (224, 224))
best_test_action_index = numpy.argmax(top)
best_test_action = (y[best_test_action_index], x[best_test_action_index])
print(numpy.max(top))
print(inverse_heights[best_test_action])
print(test_mask_rot[best_test_action])
print(best_test_action)

fig, ax = plt.subplots(1)

# ax.imshow(masked_test_color_heightmap)
# ax.imshow(inverse_heights, cmap='gray')
ax.imshow(test_color_heightmap)
# ax.imshow(test_mask)
ax.add_patch(Circle(best_test_action))

In [ ]:
scale_factor = 0.002
top_scale_factor = scale_factor * 10
points = numpy.stack((x,y,numpy.array(top)/top_scale_factor), -1) * scale_factor
print(points.shape)

rgb = (numpy.stack(colors) * 255).astype('uint8')

ply = Ply(points, rgb)

ply.write("figures/fig7/test_cc_figure_7_" + policy_name + ".ply")
# ply.write("figures/fig7/test_cc_figure_7_stack.ply")
# ply.write("figures/fig7/test_cc_figure_7_unstack.ply")
# ply.write("figures/fig7/test_cc_figure_7_square.ply")


# write_xyz_rgb_as_ply(points, blended, "figures/fig7/test_cc_figure_7_row.ply")

In [ ]:
# TODO:
# Check rotation
# Tighter mask - N/A
# Subtract minimum nonzero value - done
# Try adding the match distance
# Save image files for figures - done
# Refactor